In [1]:
# Add client package to sys path

CLIENT_PATH = '/home/jovyan/client'

import sys

if CLIENT_PATH not in sys.path:
    sys.path.append(CLIENT_PATH)

In [2]:
# Add mlapi/mlapi package to sys path for being able to load mlflow models.

MLAPI_PATH = '/home/jovyan/mlapi/mlapi'

import sys

if MLAPI_PATH not in sys.path:
    sys.path.append(MLAPI_PATH)

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np

# Sample dataset

In [5]:
import pandas as pd

In [6]:
schema = {
    'Features': [
        {
            'FeatureName': 'target',
            'FeatureType': 'float'
        },
        {
            'FeatureName': 'group_id__0',
            'FeatureType': 'string'
        },
        {
            'FeatureName': 'group_id__1',
            'FeatureType': 'string'
        },
        {
            'FeatureName': 'timestamp',
            'FeatureType': 'timestamp'
        }
    ]
}

In [7]:
df = pd.read_csv('data/target.csv')
df = df.rename(
    columns={
        'store': 'group_id__0',
        'item': 'group_id__1', 
        'date': 'timestamp', 
        'sales': 'target'
    }
)

In [8]:
dtypes = {
    'target': float,
    'group_id__0': str,
    'group_id__1': str,
}
df = df.astype(dtypes)
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Train-test split

In [9]:
from mooncake.model_selection import train_test_split

In [10]:
# Split configuration

test_start = '2017-09-01'
test_end = '2017-12-01'
date_column='timestamp'
freq='W'
sequence_length=5


# Split

X_train, X_test = train_test_split(
    df, test_start, test_end, date_column, 
    freq, sequence_length=sequence_length)

# Create session instance

In [11]:
# The `session` instance allows us to create clients for any available service.

from client.session import Session

session = Session()

In [12]:
# List available services

session.get_available_services()

['minio', 'forecast', 'login']

# Login service

In [13]:
login_client = session.create_client(
    service_name='login'
)

In [14]:
login_response = login_client.login()

Username:  johndoe
Password:  ······


In [15]:
# Collect access token from login response.

access_token = login_response.json()['access_token']

In [16]:
# Since this access token will be used in all remaining client requests,
# let us set it in the current `session` instance, so we dont worry no more about it.

session.set_access_token(access_token)

# Minio client

Minio client allows to write datasets in cloud buckets for later use with the ***forecast*** service.

In [17]:
minio_client = session.create_client(
    service_name='minio', 
    is_secure=False, 
)

In [17]:
# Write TRAIN dataset to minio bucket.

minio_client.create_dataset(
    dataset_group_name='sample_group', dataset_name='X_train',
    data=X_train, schema=schema, dataset_type='target')

{'endpoint': 'minio:9000', 'access_key': 'johndoe', 'secret_key': 'password', 'secure': False}


In [18]:
# Write TEST dataset to minio bucket.

minio_client.create_dataset(
    dataset_group_name='sample_group', dataset_name='for_inference/X_test',
    data=X_test, schema=schema, dataset_type='target')

# Forecast client

In [19]:
forecast_client = session.create_client(
    service_name='forecast', 
    is_secure=False
)

In [22]:
forecast_config = {
    'task_name': 'sample_training_task',
    'dataset_group_name': 'sample_group',
    'dataset_name': 'X_train',
    'algorithm': 'seq2seq',
    'forecast_horizon': 10,
    'perform_hpo': False,
    'inference_folder': 'for_inference'
}

http = forecast_client.create_predictor(**forecast_config)

In [23]:
# Inspect http response.
# Notice it contains the task UUID under the key 'task_id'.

http.json()

{'task_id': '9cbcca42-5f17-459a-8361-57960d7e723e', 'status': 'Processing'}

# MlFlow

In [24]:
import mlflow
import numpy as np

In [46]:
run_id =  mlflow.search_runs(filter_string='tags.mlflow.runName = "sample_training_task"').iloc[0].run_id

# Register model

In [47]:
model_name = "model A"
model_version = mlflow.register_model(f"runs:/{run_id}/sample_training_task", model_name)

Successfully registered model 'model A'.
2022/05/17 11:50:33 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: model A, version 1
Created version '1' of model 'model A'.


In [48]:
from mlflow.tracking import MlflowClient
 
client = MlflowClient()
client.transition_model_version_stage(
  name=model_name,
  version=model_version.version,
  stage="Production",
)

<ModelVersion: creation_timestamp=1652788233781, current_stage='Production', description='', last_updated_timestamp=1652788237224, name='model A', run_id='b9a80f8e609446f197c7bbb813cfedb3', run_link='', source='s3://pipelines/0/b9a80f8e609446f197c7bbb813cfedb3/artifacts/sample_training_task', status='READY', status_message='', tags={}, user_id='', version='1'>

# Load model

In [30]:
# Load model

model_name = "model B"
model = mlflow.pyfunc.load_model(f"models:/{model_name}/production")

In [33]:
model._model_impl.context.artifacts['inference']

'/tmp/tmpzxx64kx2/artifacts/.'